In [ ]:
import os

os.chdir("/home/joosep/particleflow")

In [ ]:
import tensorflow as tf
import numpy as np
import sys

sys.path.append("/home/joosep/particleflow/mlpf")
sys.path.append("/home/joosep/particleflow/hep_tfds/")
import tfmodel.model
import tfmodel.data
import tfmodel.model_setup

import yaml
import matplotlib.pyplot as plt
import matplotlib

import pandas
import networkx
import glob

from matplotlib import cm
import mplhep

mplhep.style.use("CMS")

In [ ]:
def sample_label(ax, x=0.01, y=0.93):
    plt.text(x, y, "$t\\bar{t}$ events", ha="left", transform=ax.transAxes, size=20)

In [ ]:
with open(
    "/home/joosep/particleflow/experiments/all_data_cms-best-of-asha-scikit_20211026_042043_178263.workergpu010/config.yaml"
) as f:
    config = yaml.safe_load(f)
config["setup"]["multi_output"] = True
config["parameters"]["debug"] = True

In [ ]:
model = tfmodel.model_setup.make_gnn_dense(config, tf.float32)

In [ ]:
config["datasets"]["cms_pf_ttbar"]

In [ ]:
cds = config["dataset"]

config["datasets"]["cms_pf_ttbar"]["data_dir"] = "/home/joosep/tensorflow_datasets/"
config["datasets"]["cms_pf_ttbar"]["batch_per_gpu"] = 1
ds_val, ds_info = tfmodel.utils.get_heptfds_dataset("cms_pf_ttbar", config, 1, "test", 100)

In [ ]:
ret = model.build((1, 6400, 25))
# model.set_trainable_classification()
model.load_weights(
    "/home/joosep/particleflow/experiments/all_data_cms-best-of-asha-scikit_20211026_042043_178263.workergpu010/weights/weights-200-0.074496.hdf5"
)

In [ ]:
model.summary()

In [ ]:
for X, y, w in ds_val:
    X = tf.expand_dims(X, axis=0)
    X_val = X.numpy()
    ret = model.predict_on_batch(X)
    break

In [ ]:
def get_bin_index(bs):
    bin_index = []

    for ielem in range(6400):
        if X_val[0, ielem, 0] != 0:
            for ibin in range(bs.shape[0]):
                if ielem in bs[ibin]:
                    bin_index.append(ibin)
                    break
        else:
            break
    return bin_index

In [ ]:
def plot_binning_in_layer(layer_name):
    msk = X_val[0][:, 0] != 0
    eta = X_val[0][msk, 2]
    phi = X_val[0][msk, 3]
    typ = X_val[0][msk, 0]
    energy = X_val[0][msk, 4]

    evenly_spaced_interval = np.linspace(0, 1, ret[layer_name]["bins"].shape[1])
    colorlist = [cm.Dark2(x) for x in evenly_spaced_interval]
    bin_idx = get_bin_index(ret[layer_name]["bins"][0])

    plt.figure(figsize=(10, 10))
    mplhep.cms.label("Preliminary", data=False, loc=0, rlabel="Run 3 (14 TeV)")
    ax = plt.axes()
    sc = plt.scatter(eta, phi, c=[colorlist[bi] for bi in bin_idx], marker=".", s=energy)
    plt.legend(*sc.legend_elements("sizes", num=5), ncol=4, loc=1, title="PFElement energy [GeV]", frameon=False)
    plt.xlabel("PFElement $\eta$")
    plt.ylabel("PFElement $\phi$")
    # plt.title("Binning in {}".format(layer_name))
    # cms_label(ax)
    sample_label(ax, x=0.05)
    plt.ylim(-4.4, 4.4)
    plt.text(
        0.5,
        0.05,
        "Each point corresponds to a PFElement in a simulated event.\nUnique colors correspond to the bin assignment in this layer.",
        ha="center",
        va="center",
        transform=ax.transAxes,
        fontsize=15,
    )
    plt.savefig("bins_{}.pdf".format(layer_name), bbox_inches="tight")
    plt.savefig("bins_{}.png".format(layer_name), bbox_inches="tight", dpi=300)

In [ ]:
plot_binning_in_layer("cg_0")

In [ ]:
plot_binning_in_layer("cg_1")

In [ ]:
plot_binning_in_layer("cg_2")

In [ ]:
plot_binning_in_layer("cg_energy_0")

In [ ]:
plot_binning_in_layer("cg_energy_1")

In [ ]:
import math


def plot_dms(dms):
    num_plots = len(dms)
    sqrt_num_plots = int(math.sqrt(num_plots))
    fig = plt.figure(figsize=(sqrt_num_plots * 4, sqrt_num_plots * 4))
    mplhep.cms.label("Preliminary", data=False, loc=0, rlabel="Run 3 (14 TeV)")
    for i in range(min(len(dms), num_plots)):
        ax = plt.subplot(sqrt_num_plots, sqrt_num_plots, i + 1)
        plt.axes(ax)
        plt.imshow(dms[i], interpolation="none", norm=matplotlib.colors.Normalize(vmin=0, vmax=1), cmap="Blues")
        # plt.colorbar()
        plt.title("bin {}".format(i))
        # plt.xlabel("elem index $i$")
        # plt.ylabel("elem index $j$")
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout()

In [ ]:
for layer in ["cg_0", "cg_1", "cg_energy_0", "cg_energy_1"]:
    dm_vals = ret[layer]["dm"].flatten()
    plt.hist(dm_vals[dm_vals != 0], bins=np.linspace(0, 1, 100), density=True, lw=2, histtype="step", label=layer)
plt.yscale("log")
plt.legend(loc="best", frameon=False, ncol=2)
plt.xlabel("Element-to-element distance")
plt.ylabel("Number of elements")

plt.savefig("dm.pdf", bbox_inches="tight")
plt.savefig("dm.png", bbox_inches="tight", dpi=300)

In [ ]:
plot_dms(dmn[:4])

In [ ]:
dmn = ret["cg_0"]["dm"][0, :, :, :, 0]
plot_dms(dmn)
plt.tight_layout()
plt.suptitle("Learned adjacency, cg_0", y=1.01)
plt.savefig("dm_cg_0.pdf", bbox_inches="tight")
plt.savefig("dm_cg_0.png", bbox_inches="tight", dpi=300)

In [ ]:
dmn = ret["cg_1"]["dm"][0, :, :, :, 0]
plot_dms(dmn)
plt.suptitle("Learned adjacency, cg_1", y=1.01)
plt.savefig("dm_cg_1.pdf", bbox_inches="tight")
plt.savefig("dm_cg_1.png", bbox_inches="tight", dpi=300)

In [ ]:
dmn = ret["cg_energy_0"]["dm"][0, :, :, :, 0]
plot_dms(dmn)
plt.suptitle("Learned adjacency, cg_energy_0", y=1.01)
plt.savefig("dm_cg_energy_0.pdf", bbox_inches="tight")
plt.savefig("dm_cg_energy_0.png", bbox_inches="tight", dpi=300)

In [ ]:
dmn = ret["cg_energy_1"]["dm"][0, :, :, :, 0]
plot_dms(dmn)
plt.suptitle("Learned adjacency, cg_energy_1", y=1.01)
plt.savefig("dm_cg_energy_1.pdf", bbox_inches="tight")
plt.savefig("dm_cg_energy_1.png", bbox_inches="tight", dpi=300)